Tutorials
---------
This tutorial introduction will give example scripts on how to: 
- **prepare a multisoap average kernel**
- **do FPS sampling**
- **fit a model with KRR**
- **perform kPCA**
- **and calculate reaction energies from predicted atomization energies**

In [1]:
# Load modules
import ase.io
from mltools.gap import Gap
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from MLinCRS.krr import get_krr_coeff, get_prediction, grid_search_1d_validation_set
from MLinCRS.kPCA import kPCA
from MLinCRS.reactiontools import read_reaction_network_g0, append_energy_4_rearrangements

Multisoap kernel matrix:
------------------------
The following script prepares an average multisoap kernel for 500 molecules using the **calc_kernel_matrix_soap()** method from mltools.

To this end, we have to define a descriptor string, which specifies the parameters for the SOAP descriptor. Descriptors are created using the quippy code. Further instructions how to specify the descriptor string are given on https://libatoms.github.io/GAP/quippy-descriptor-tutorial.html. In this example we will use two SOAP representaions with different `cutoff` and `atom_sigma` parameters to account for short and long range effects. 


In [2]:
# Define parameters for short range soap
cutoff = 2.
atom_sigma = 0.3
# Descriptor string for short range soap
desc_str_2 = f"soap cutoff={cutoff} l_max=8 n_max=8 atom_sigma={atom_sigma} n_species=3 species_Z={{1 6 8}} n_Z=3 Z={{1 6 8}}"

# Define parameters for long range soap
cutoff = 4.
atom_sigma = 0.6
desc_str_4 = f"soap cutoff={cutoff} l_max=8 n_max=8 atom_sigma={atom_sigma} n_species=3 species_Z={{1 6 8}} n_Z=3 Z={{1 6 8}}"

Next, the molecules are loaded for which the descriptors are to be calculated:

In [3]:
# Load molecules
m = ase.io.read('./data_store/molecules/mols_ntrain_500_uff_k_int_fps_int.xyz@:')

We initialize a Gap instance and set the list of molecules as the variable **atoms_other**. In order to calculate the atomic descriptors for the atoms in each molecule, the quippy code needs lattices around them. This can easily be achieved with the gap method **set_lattices()**.

In [4]:
# Initialize the Gap instance
gap = Gap()
gap.atoms_other = m
gap.set_lattices(50, 'other')

Descriptors are now generated via the method **get_descriptors()** for SOAP representations with cutoff 2 Å and 4 Å. The descriptors are stored in lists. For every molecule you get an matrix with the same number of descriptors as atoms occur in the molecule.

In [5]:
# Descriptors
descs_2 = gap.get_descriptors('other', desc_str_2)
descs_4 = gap.get_descriptors('other', desc_str_4)

Now we can prepare the average kernel via the method **calc_kernel_matrix_soap()**. Therefore the parameter `kernel_type` has to be set to the keyword *average* (which is also the default) and we normalize the kernel matrix so that K(i,j) = K(i,j)/sqrt(K(i,i) K(j,j)) by setting `normalize` to *True* (also default). Further parameters can be set e.g. `zeta`, `ncores`, `verbose`, ... that are documented in the mltools source code. 

If you want to calculate the sum kernel instead, the parameter `kernel_type` has to set to *sum*. Since the sum kernel is an extensive kernel the individual kernel elements are not normalized, which is the default.     

In [6]:
kernel_matrix_2 = gap.calc_kernel_matrix_soap(
    descs_2,
    header=desc_str_2,
    zeta=2.,
    kernel_type='average',
    normalize=True,
    ncores=2,
    verbose=True,
    destination='./data_store/kernels/test_kernel_2.txt'
)
kernel_matrix_4 = gap.calc_kernel_matrix_soap(
    descs_4,
    header=desc_str_4,
    zeta=2.,
    kernel_type='average',
    normalize=True,
    destination='./data_store/kernels/test_kernel_4.txt'
)


Calculate the average kernel for 500 molecules
2020-09-12 09:34:08.975711 Kernel element (diagonal) :     10/500 (for normalization)
2020-09-12 09:34:08.986321 Kernel element (diagonal) :     20/500 (for normalization)
2020-09-12 09:34:08.997846 Kernel element (diagonal) :     30/500 (for normalization)
2020-09-12 09:34:09.007346 Kernel element (diagonal) :     40/500 (for normalization)
2020-09-12 09:34:09.016684 Kernel element (diagonal) :     50/500 (for normalization)
2020-09-12 09:34:09.026853 Kernel element (diagonal) :     60/500 (for normalization)
2020-09-12 09:34:09.041611 Kernel element (diagonal) :     70/500 (for normalization)
2020-09-12 09:34:09.054626 Kernel element (diagonal) :     80/500 (for normalization)
2020-09-12 09:34:09.064025 Kernel element (diagonal) :     90/500 (for normalization)
2020-09-12 09:34:09.067700 Kernel element (diagonal) :     100/500 (for normalization)
2020-09-12 09:34:09.073507 Kernel element (diagonal) :     110/500 (for normalization)
2020-

2020-09-12 09:34:11.783199 Kernel element (off-diagonal) : 4656/124750 (3.7323 %)
2020-09-12 09:34:11.848533 Kernel element (off-diagonal) : 4851/124750 (3.8886 %)
2020-09-12 09:34:11.920126 Kernel element (off-diagonal) : 5050/124750 (4.0481 %)
2020-09-12 09:34:11.988533 Kernel element (off-diagonal) : 5253/124750 (4.2108 %)
2020-09-12 09:34:12.059597 Kernel element (off-diagonal) : 5460/124750 (4.3768 %)
2020-09-12 09:34:12.144051 Kernel element (off-diagonal) : 5671/124750 (4.5459 %)
2020-09-12 09:34:12.235656 Kernel element (off-diagonal) : 5886/124750 (4.7182 %)
2020-09-12 09:34:12.301708 Kernel element (off-diagonal) : 6105/124750 (4.8938 %)
2020-09-12 09:34:12.368837 Kernel element (off-diagonal) : 6328/124750 (5.0725 %)
2020-09-12 09:34:12.444751 Kernel element (off-diagonal) : 6555/124750 (5.2545 %)
2020-09-12 09:34:12.528927 Kernel element (off-diagonal) : 6786/124750 (5.4397 %)
2020-09-12 09:34:12.606209 Kernel element (off-diagonal) : 7021/124750 (5.6281 %)
2020-09-12 09:34

2020-09-12 09:34:23.617421 Kernel element (off-diagonal) : 43365/124750 (34.7615 %)
2020-09-12 09:34:23.813792 Kernel element (off-diagonal) : 43956/124750 (35.2353 %)
2020-09-12 09:34:23.956275 Kernel element (off-diagonal) : 44551/124750 (35.7122 %)
2020-09-12 09:34:24.128044 Kernel element (off-diagonal) : 45150/124750 (36.1924 %)
2020-09-12 09:34:24.308984 Kernel element (off-diagonal) : 45753/124750 (36.6758 %)
2020-09-12 09:34:24.498397 Kernel element (off-diagonal) : 46360/124750 (37.1623 %)
2020-09-12 09:34:24.707135 Kernel element (off-diagonal) : 46971/124750 (37.6521 %)
2020-09-12 09:34:24.873594 Kernel element (off-diagonal) : 47586/124750 (38.1451 %)
2020-09-12 09:34:25.049242 Kernel element (off-diagonal) : 48205/124750 (38.6413 %)
2020-09-12 09:34:25.243721 Kernel element (off-diagonal) : 48828/124750 (39.1407 %)
2020-09-12 09:34:25.432033 Kernel element (off-diagonal) : 49455/124750 (39.6433 %)
2020-09-12 09:34:25.660925 Kernel element (off-diagonal) : 50086/124750 (40.

2020-09-12 09:34:48.099142 Kernel element (off-diagonal) : 120295/124750 (96.4289 %)
2020-09-12 09:34:48.453946 Kernel element (off-diagonal) : 121278/124750 (97.2168 %)
2020-09-12 09:34:48.777098 Kernel element (off-diagonal) : 122265/124750 (98.0080 %)
2020-09-12 09:34:49.101565 Kernel element (off-diagonal) : 123256/124750 (98.8024 %)
2020-09-12 09:34:49.418219 Kernel element (off-diagonal) : 124251/124750 (99.6000 %)
2020-09-12 09:34:49.862372 Kernel element (off-diagonal) : 124750/124750 (100.0000 %)
Normalize kernel matrix ...
... ok!

Save kernel matrix as ./data_store/kernels/test_kernel_2.txt ...
... ok!

Calculate the average kernel for 500 molecules
2020-09-12 09:35:51.988207 Kernel element (diagonal) :     500/500 (for normalization)
Normalize kernel matrix ...
... ok!

Save kernel matrix as ./data_store/kernels/test_kernel_4.txt ...
... ok!



To obtain a multisoap kernel the individual kernel matrices are averaged. Note for the sum kernel individual kernels are just summed up (kernel_matrix = kernel_matrix_2 + kernel_matrix_4). 

In [ ]:
kernel_matrix_2 = np.loadtxt('./data_store/kernels/test_kernel_2.txt')
kernel_matrix_4 = np.loadtxt('./data_store/kernels/test_kernel_4.txt')
kernel_matrix = 0.5*(kernel_matrix_2 + kernel_matrix_4)

FPS:
----
Training set selection can be done using the fartherst point sampling (FPS) method to obtain a representative set. To this end, we use the **find_farthest()** method to generate a sequence that can be applied to the molecules array. To use this method we need the respective distance matrix from the kernel matrix. The distance matrix can be calculated via the **calc_distance_matrix()** method. For doing the FPS we have to specify the `seeds` and `number` parameters. The `seeds` parameter can be an interger or list of integers that serve as starting samples. The parameter `number` defines how many structures are to be selected. Once the sequence is generated one can apply it to the molecules array. Note in this example molecules are already ordered according to the fps sampling. For that reason the resulting sequence ranges from 0 to 499.

In [ ]:
# Calculate distance matrix for farthest point sampling
dist_matrix = gap.calc_distance_matrix(kernel_matrix)

# Do FPS
seeds = [0,]
number = 500
seq = gap.find_farthest(dist_matrix, seeds, number)
print(seq)
m = np.asarray(m, dtype='object')[seq]

Fits:
-----
In order to predict molecular energies one can use Kernel Ridge Regression and the multisoap average kernel. As a first step we grep the atomization energies (AE) from the atoms objects in ase. Since the average kernel is an intensive kernel matrix, the quantity to fit also has to be intensive. We therefore fit on the AE per atom. To do this, the number of atoms N in each molecule has to be determined. Note: If the sum kernel is used one should fit to the AE.  

In [ ]:
# Get AE and number of atoms in a molecule
AE = np.asarray([mol.info['AE'] for mol in m])
N = np.asarray([float(len(mol.get_chemical_symbols())) for mol in m]) 
# Divide AE through N to get an intensive quantity
AE_N = AE/N
# Enable or disable here to display AE/N
# AE_N

The KRR coeffients alpha are obtained via the function **get_krr_coeff()**. For the intensive fits we applied a mean correction to the AE per atom. This is done by setting the mean of the training energies to the paramter `mc`. The parameter `default_sigma` represents the regularization parameter. It is a single value. If one wants to set a regularization parameter for every individual molecule (vector) one has to set the `y_sigma` parameter. In this case, `default_sigma` is overwritten. `default_sigma` can also be determined via grid search with the function **grid_search_1d_validation_set()**. This is shown in the tutorial below.    

In [ ]:
# Training
alpha = get_krr_coeff(kernel_matrix, AE_N, default_sigma=0.001, mc=np.mean(AE_N))
# alpha

The quantities of interest can be obtained via the function **get_prediction()**, which uses the kernel matrix, the alpha coefficients and the mean energy (if a mean correction is applied) of the training data as inputs. With the obtained energies, errors like the mean absolute error can be calculated.

In [ ]:
# Prediction of training data   
y = get_prediction(kernel_matrix, alpha, mc=np.mean(AE_N))
# y
print(f"MAE training set: {mean_absolute_error(AE, y*N): .3f} eV")

Prediction of unknown data samples:
-----------------------------------
In order to predict unknown data, a new kernel matrix has to be prepared. This can be done via the method **calc_soap_kernel_between_sets()**. This method uses two lists of descriptor matrices for two molecule sets (e.g. training and prediction set.) Therefore we have now to create the list of descriptors for the molecules to be predicted. This is done in a similar way as shown in the beginning of the tutorial.  

In [ ]:
# Prediction
# Load molecules to be predicted
m_pred = ase.io.read('./data_store/molecules/mols_validation.xyz@:')
# Set molecules to the variable atoms_other and set lattices around them. 
gap.atoms_other = m_pred
gap.set_lattices(50, 'other')


In [ ]:
# Calculate the two different descriptors for the multisoap kernel.
descs_pred_2 = gap.get_descriptors('other', desc_str_2)
kernel_pred_2 = gap.calc_soap_kernel_between_sets(
    descs_2,  # List with descriptors from the training set
    descs_pred_2,  # List with descriptors from the test set
    zeta=2.,
    kernel_type='average',
)
descs_pred_4 = gap.get_descriptors('other', desc_str_4)
kernel_pred_4 = gap.calc_soap_kernel_between_sets(
    descs_4,  # List with descriptors from the training set
    descs_pred_4,  # List with descriptors from the test set
    zeta=2.,
    kernel_type='average',
)
# Calculate the multisoap kernel
kernel_pred = 0.5*(kernel_pred_2 + kernel_pred_4)
np.savetxt('./data_store/kernels/prediction_kernel.txt', kernel_pred, header=f"{desc_str_2}, {desc_str_4}")

In [ ]:
kernel_pred = np.loadtxt('./data_store/kernels/prediction_kernel.txt')

Predict the energies and calculate the MAE.

In [ ]:
# Predict energies as y_pred
y_pred = get_prediction(kernel_pred, alpha, mc=np.mean(AE_N))

# Grep AE and number of molecules from list with ase atoms objects
AE_pred = np.asarray([mol.info['AE'] for mol in m_pred])
N_pred = np.asarray([float(len(mol.get_chemical_symbols())) for mol in m_pred])

# Get MAE of predictions (recalculate AE/N in AE by multipling them with N)
print(f"MAE training set: {mean_absolute_error(AE_pred, y_pred*N_pred): .3f} eV")

Define regularization parameter with grid search:
-------------------------------------------------
To determine the regularization parameter sigma in KRR the **grid_search_1d_validation_set()** function can be used. To this end, one has to define a grid with different sigma paramaters that should be checked. Further inputs to this function are the training kernel matrix, the matrix used to do the predictions for the validation set and the respective AE per atoms (for both training and validation sets) as well as the mean of the training set energies if mean correction is applied. The results (hypersurfaces) are stored in a matrix that contains the following three columns: 1.) sigma values from the grid, 2.) RMSE train [eV], 3.) RMSE validation [eV] . This matrix is saved as *hypersurface_data.txt* in the directory *'./Results_validation_set'* by default. 

In [ ]:
# Define grid with sigma values
sigmas = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 0.1, 0.5]

# Run the grid search with grid_search_1d_validation
grid_search_1d_validation_set(kernel_matrix, kernel_pred, AE_N, AE_pred/N_pred, sigmas, mc=np.mean(AE_N), destination='./data_store/Results_validation_set') 

The resulting hypersurfaces can be plotted and the sigma which has the lowest RMSE of the training set is used.  

In [ ]:
# Load surface
surface = np.loadtxt('./data_store/Results_validation_set/hypersurface_data.txt')
#plot surface 
plt.plot(np.log10(surface[:, 0]), surface[:, 1], 'v', ls='--', label='train',c='royalblue')
plt.plot(np.log10(surface[:, 0]), surface[:, 2], 'D', ls='-.', label='validate',c='crimson')
plt.axvline(np.log10(surface[np.argmin(surface[:, 2]), 0]), c='k', label=r'$\sigma$', ls=':')
plt.xlabel(r'log($\sigma$)')
plt.ylabel(r'AE/$N$ [eV]')
plt.legend()

print(f"Regularization parameter is: {surface[np.argmin(surface[:, 2]), 0]}")
print(f"{'sigma':7s} {'RMSE train [eV]':10s} {'RMSE validation [eV]':10s}")
for line in surface:
    print(f"{line[0]:0.5f} {line[1]:0.5f} {'':8s}{line[2]:0.5f}")

Kernel PCA:
-----------
In this script we show how to do a kernel PCA using the average kernel from above. To this end, the **kPCA()** function is used, which uses the kernel matrix as an input. The function returns the first three PCs with the three highest eigenvalues. These principal components can be ploted as shown below.

In [ ]:
PC1, PC2, PC3, eigenvals = kPCA(kernel_matrix)

In [ ]:
print(eigenvals)

In [ ]:
plt.scatter(PC1, PC2, c=AE_N)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.colorbar()

Or you can plot it in 3D:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(PC1, PC2, s=20, c=AE_N)
ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.set_zlabel('PC 3')

Reaction energies:
------------------
In this tutorial we use a very small reaction network of only three reactions as a showcase to illustrate how reaction energies can be calculated. The reaction network contains only dissociation and rearrangement reactions of type A -> B + C and A -> B. Note that only this type of network (e.g. like Rad-6-RE) can be analyzed with the underlying code. 

The first two steps should be familiar, since we are now creating the kernel matrix for the 8 molecules to be predicted with the 500 molecules from the training set and predict the AE. 

In [ ]:
# Load molecules included in the reaction network
m_RE = ase.io.read('./data_store/molecules/mols_reaction.xyz@:')
# Set molecules to the variable atoms_other and set lattices around them. 
gap.atoms_other = m_RE
gap.set_lattices(50, 'other')

# Calculate the two different descriptors for the multisoap kernel.
descs_RE_2 = gap.get_descriptors('other', desc_str_2)
kernel_RE_2 = gap.calc_soap_kernel_between_sets(
    descs_2,  # List with descriptors from the training set
    descs_RE_2,  # List with descriptors from the test set
    zeta=2.,
    kernel_type='average',
)
descs_RE_4 = gap.get_descriptors('other', desc_str_4)
kernel_RE_4 = gap.calc_soap_kernel_between_sets(
    descs_4,  # List with descriptors from the training set
    descs_RE_4,  # List with descriptors from the test set
    zeta=2.,
    kernel_type='average',
)
kernel_RE = 0.5*(kernel_RE_2 + kernel_RE_4)

In [ ]:
# Prediction of AE/N for molecules in the network
y_RE = get_prediction(kernel_RE, alpha, mc=np.mean(AE_N))

# Grep N and AE from list with ase atoms objects
N_RE = np.asarray([float(len(mol.get_chemical_symbols())) for mol in m_RE]) 
AE_RE = np.asarray([mol.info['AE'] for mol in m_RE])

# Get MAE for AE
print(f"MAE RE set: {mean_absolute_error(AE_RE, y_RE*N_RE): .3f} eV")

In the next step we detect how many molecules are involved in the network or rather what is the largest id of the molecules in the network. This is done because it is also possible that you have a database and not all molecules are part of the network. Therefore you need the id and not just the number of molecules. Usually both numbers are however the same. With this in hand, we apply the function **read_reaction_network_g0()**. This functions returns the indices for the educts and products of the molecules (ids) that can be applied to the predicted AE arrays. We further use the function **append_energy_4_rearrangements()** to also account for rearrangement reactions by appending an entry with 0 eV to the predicted AE array.  

In [ ]:
# Get molecule id's
ids = [mol.info['id'] for mol in m_RE]

# Append zero energy value atomization energies 
AE_RE_new = append_energy_4_rearrangements(y_RE*N_RE)
print(AE_RE_new)

# Get indices for molecules involved in the network
# Reaction network of type A -> B + C 
# e = indices of A, p1 = indices of B, p2 = indices of C 
e, p1, p2, reaction_energies = read_reaction_network_g0('./data_store/network/test_RE.txt', np.max(ids))



The reaction energies can now be calculated by using the indices and substracting the educt energies from the product energies.

In [ ]:
RE_calc = (AE_RE_new[p1]+ AE_RE_new[p2])-AE_RE_new[e]
print(f"The obtained reaction energies are: {RE_calc}")
print(f"MAE for three reactions: {mean_absolute_error(reaction_energies, RE_calc):0.3f} eV")